In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------


Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 11

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v08"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v08 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V11/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V11/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2019M06D27 UTC 09:48


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//155562_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//142625_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//223940_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//127443_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//172497_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//153021_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//122784_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111011_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//181976_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115801_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//131902_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//157807_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//145102_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//132895_V11.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//224002_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//172524_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//155571_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//232160_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//132930_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//142634_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//162300_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115808_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//122820_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//145109_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//131909_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//158116_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111019_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//153028_V11.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//132947_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//158142_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//162444_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//132170_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//183000_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//116005_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111045_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//145207_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//153035_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//155579_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//172542_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//127468_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//142643_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//225111_V11.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//232302_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//132955_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117101_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//155616_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//142651_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//132224_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//172549_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//158150_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//122855_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//145404_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//127485_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//153043_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//211016_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//225118_V11.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//132241_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//225160_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//172557_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//142658_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//145601_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111088_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//153050_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//211034_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//127493_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//162469_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//155634_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//132973_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117109_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//158218_V11.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//726013_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//642454_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//742988_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//712605_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//772347_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911104_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//822476_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914706_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//742441_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//753306_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//812204_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//831954_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//782701_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//652207_V11.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//742996_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//723074_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//642462_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//822493_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911214_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//742448_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//812302_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//831970_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//753404_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//726030_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//782708_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//662908_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//652304_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//713000_V11.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//726074_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//742483_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//753502_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//772407_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//743006_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//812309_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//652402_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//713225_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//822512_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//664024_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//642471_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//832106_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911261_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//782806_V11.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//822519_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//652500_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911270_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//832422_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//782930_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//726083_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//772605_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//723118_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//664043_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//753610_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//713260_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//742491_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911281_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//812472_V11.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//731110_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//751402_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//832441_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//782994_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//812490_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//772804_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//753628_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911402_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//742510_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//822548_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//751403_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//832442_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//731120_V11.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//782995_V11.csv
/volumes/data/Y2018M

In [14]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [15]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111011_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111012_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111013_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111014_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111015_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111030_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111016_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111017_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//111018_V11.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112140_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112117_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112150_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112160_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112180_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112190_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112170_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112300_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//112400_V11.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114444_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114445_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114446_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114447_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114448_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114450_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114460_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114449_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//114470_V11.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115424_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115425_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115426_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115427_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115428_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115429_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115430_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115440_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//115451_V11.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117106_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117107_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117108_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117109_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117201_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117202_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117203_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117204_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//117205_V11.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862070_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862080_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862091_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862092_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862093_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862094_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862095_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862096_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//862097_V11.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911269_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911270_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911281_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911282_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911283_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911284_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911285_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911286_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911287_V11.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//911990_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//912102_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//912101_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//912103_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//912104_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//912106_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//912108_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//912107_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//912109_V11.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913410_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913420_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913430_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913441_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913442_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913443_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913444_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913445_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//913446_V11.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914205_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914203_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914207_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914206_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914208_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914300_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914209_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914401_V11.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V11//914402_V11.csv [Content-Type=text/

In [16]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:25:04.077634


Previous runs:  
0:42:01.416597  
0:42:14.565295  
0:31:40.728932  
0:23:35.132388

